In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
main_path = Path('..').resolve()
sys.path.append(str(main_path))

import seaborn as sns
from src.dataset import MetaStockDataset
from src.utils import ARGProcessor
import torch
import matplotlib.pyplot as plt
torch.__version__

'1.11.0+cu113'

In [2]:
setting_file = Path('.') / 'kdd.yml'

meta_args = ARGProcessor(setting_file=setting_file)
data_kwargs = meta_args.get_args(cls=MetaStockDataset)

meta_train = MetaStockDataset(meta_type='train', **data_kwargs)
meta_valid_time = MetaStockDataset(meta_type='valid-time', **data_kwargs)
meta_valid_stock = MetaStockDataset(meta_type='valid-stock', **data_kwargs)
meta_valid_mix = MetaStockDataset(meta_type='valid-mix', **data_kwargs)
meta_test_time = MetaStockDataset(meta_type='test-time', **data_kwargs)
meta_test_stock = MetaStockDataset(meta_type='test-stock', **data_kwargs)
meta_test_mix = MetaStockDataset(meta_type='test-mix', **data_kwargs)

Processing data and candidates for test-mix: 100%|██████████| 5/5 [00:00<00:00, 81.75it/s]


In [4]:
all_data = meta_train.generate_tasks()
all_data

StockDataDict(T=15, numpy)
- query: (64, 1, 15, 11)
- query_labels: (64,)
- support: (64, 20, 15, 11)
- support_labels: (1280,)

In [5]:
all_data.to('cpu')
all_data

StockDataDict(T=15, tensor.cpu)
- query: torch.Size([64, 1, 15, 11])
- query_labels: torch.Size([64])
- support: torch.Size([64, 20, 15, 11])
- support_labels: torch.Size([1280])

In [ ]:
all_data.numpy()
all_data

In [ ]:
all_data['query'][..., 0].reshape(-1).shape

In [ ]:
meta_train.meta_type

In [ ]:
def draw_density(ds):
    all_data = ds.generate_tasks()
    fig, axes = plt.subplots(11, 2, figsize=(10, 16))
    for i in range(11):
        for t in range(2):
            
            f1_q = all_data['query'][..., i].reshape(-1)
            f1_s = all_data['support'][:, t, :, i].reshape(-1)
            sns.histplot(data=f1_q, ax=axes[i, t], color="blue", label='query', alpha=0.2)
            sns.histplot(data=f1_s, ax=axes[i, t], color="red", label='support', alpha=0.2)
            axes[i, t].legend()
            if i == 0:
                axes[i, t].set_title(f'Class: {t}')
    fig.suptitle(f'{ds.meta_type}')
    plt.tight_layout()
    plt.show()

In [ ]:
draw_density(ds=meta_train)

In [ ]:
draw_density(ds=meta_valid_time)

In [ ]:
draw_density(ds=meta_valid_stock)

data distribution

In [ ]:
from collections import Counter

def count_labels(meta_ds):
    cnts = Counter()
    for s in meta_ds.symbols:
        t = meta_ds.data[s].loc[meta_ds.candidates[s], 'label'].value_counts().to_dict()
        cnts.update(t)
    return cnts

cnt_data = {'ds': [], 'n_stock': [], 'fall': [], 'rise': []}
for ds in [meta_train, meta_valid_time, meta_valid_stock, meta_valid_mix, meta_test_time, meta_test_stock, meta_test_mix]:
    cnts = count_labels(ds)
    cnt_data['ds'].append(ds.meta_type)
    cnt_data['n_stock'].append(len(ds.symbols))
    cnt_data['fall'].append(cnts[0])
    cnt_data['rise'].append(cnts[1])
df_cnt = pd.DataFrame(cnt_data)
df_cnt

In [ ]:
# q dist
from tqdm import tqdm

def plot_q_dist(meta_dataset):
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    idx = np.arange(max(meta_dataset.q_dist.keys()))
    values = [meta_dataset.q_dist[i] if meta_dataset.q_dist.get(i) else 0 for i in idx]

    ax.bar(idx, values)
    ax.set_xlabel('Query index in labels')
    ax.set_ylabel('Count')
    ax.set_title(f'Meta Type: {meta_dataset.meta_type}')
    plt.tight_layout()
    return fig

In [ ]:
meta_train.reset_q_idx_dist()
n = 1000
for i in tqdm(range(n), total=n):
    meta_train.generate_tasks()

In [ ]:
fig = plot_q_dist(meta_train)

In [ ]:
meta_train.data.keys()

In [ ]:
meta_train.data['AAPL']

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('./test_writer')
writer.add_figure('b', fig)

---

##  Check Time is Enough

In [19]:
meta_train.n_support

10

In [30]:
def test_time(ds, n_support=10):
    window_size = 15
    ds.n_support = n_support
    cnt = 0
    for symbol in ds.symbols:
        df_stock = ds.data[symbol]
        labels_indices = ds.candidates[symbol] 
        labels_indices = labels_indices[labels_indices >= window_size]

        for i in range(len(labels_indices)):
            array = df_stock.loc[labels_indices, 'label'].loc[:(labels_indices[i])].to_numpy()
            if ds.check_condition(array):
                break
        if i == len(labels_indices)-1:
            cnt += 1
    return cnt

In [35]:
ds_list = [meta_train, meta_valid_time, meta_valid_stock, meta_valid_mix, meta_test_time, meta_test_stock, meta_test_mix]

In [40]:
for ds in ds_list:
    print(f'{ds.meta_type}({len(ds.symbols)})', test_time(ds, n_support=30))

train(35) 0
valid-time(35) 0
valid-stock(10) 0
valid-mix(10) 0
test-time(35) 0
test-stock(5) 0
test-mix(5) 0


## Data generator

In [18]:
window_size = 15
symbol = 'AAPL'
df_stock = meta_train.data[symbol]
# filter out unpossible candidates
labels_indices = meta_train.candidates[symbol] 
labels_indices = labels_indices[labels_indices >= window_size]

for i in range(len(labels_indices)):
    array = df_stock.loc[labels_indices, 'label'].loc[:(labels_indices[i])].to_numpy()
    if meta_train.check_condition(array):
        break

# satisfied condition label index | smallest support index | smallest query index
candidates = labels_indices[(i+1):]

In [ ]:
df_stock.loc[labels_indices].iloc[:10, -1:]

In [ ]:
data = dict(
    query = None,
    query_labels = None,
    support = None,
    support_labels = None,
)

q_target = np.random.choice(candidates)   # index in the dataframe
# for q_target in y_q:
    # Queries
q_idx = np.arange(len(labels_indices))[labels_indices == q_target][0]  # get the index of label data
q_end = np.array([q_target]) 
q_start = q_end - window_size
q_data, q_labels = meta_train.generate_data(df_stock, y_start=q_start, y_end=q_end)

data['query'] = q_data
data['query_labels'] = q_labels[0]  # (1,)

# Supports
s_fall, s_rise = meta_train.get_rise_fall(df_stock, labels_indices, idx=q_idx, n_select=meta_train.n_support)
s_end = np.concatenate([s_fall, s_rise])
s_start = s_end - window_size
s_data, s_labels = meta_train.generate_data(df_stock, y_start=s_start, y_end=s_end)

data['support'] = s_data
data['support_labels'] = s_labels  # (N*K,)

print()   
print(f'query index: {q_idx}({q_target}) = {df_stock.loc[q_target, "label"]}')
print(f'- start={q_start} end={q_end}')
print(f'support indices:')
print(f'- start={s_start} end={s_end}')
print(f'{df_stock.loc[s_end, "label"]}')


---

## Check queries distribution

In [ ]:
from collections import Counter
window_size = 10
def get_q_label_dist(ds):
    q_label_dist = Counter()
    for symbol in ds.symbols:
        df_stock = ds.data[symbol]
        # filter out unpossible candidates
        labels_indices = ds.candidates[symbol] 
        labels_indices = labels_indices[labels_indices >= window_size]

        for i in range(len(labels_indices)):
            array = df_stock.loc[labels_indices, 'label'].loc[:(labels_indices[i])].to_numpy()
            if ds.check_condition(array):
                break
        candidates = labels_indices[(i+1):]  # query candidates
        
        counts = df_stock.loc[candidates, 'label'].value_counts().to_dict()
        q_label_dist.update(counts)
    
    return q_label_dist

In [ ]:
q_label_dists = {'type': [], 'fall': [], 'rise': []}
for ds in [meta_train, meta_valid_time, meta_valid_stock, meta_valid_mix, 
    meta_test_time, meta_test_stock, meta_test_mix]:
    q_label_dist = get_q_label_dist(ds)
    q_label_dists['type'].append(ds.meta_type)
    q_label_dists['fall'].append(q_label_dist[0])
    q_label_dists['rise'].append(q_label_dist[1])

q_label_dists = pd.DataFrame(q_label_dists)

In [ ]:
q_label_dists

---

# Modeling

In [6]:
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from src.model import MetaModel
import math
model_kwargs = meta_args.get_args(cls=MetaModel)
model = MetaModel(**model_kwargs)

rt_attn = True
data = all_data
n_inner_step = 10

In [15]:
inner_lr = nn.Parameter(torch.FloatTensor([1.0]))

In [81]:
s_inputs = data['support']
s_labels = data['support_labels']

# Forward Encoder
s_l, s_z, kld_loss, s_attn = model.forward_encoder(s_inputs, rt_attn=rt_attn)

# initialize z', Forward Decoder
z_prime = s_z.detach()
z_prime.requires_grad_(True)
s_pred_loss, s_param_l2_loss, s_preds, parameters = model.forward_decoder(z=z_prime, l=s_l, labels=s_labels)
s_loss = s_pred_loss + model.param_l2_lambda * s_param_l2_loss

print(f's_z: {s_z.requires_grad} | z_prime = {z_prime.requires_grad}')
print(f's_z: {s_z.is_leaf} | z_prime = {z_prime.is_leaf}')

s_z: True | z_prime = True
s_z: False | z_prime = True


In [88]:
inner_optimizer = torch.optim.SGD([z_prime], lr=0.1)
inner_scheduler = torch.optim.lr_scheduler.ExponentialLR(inner_optimizer, gamma=0.9)
# inner_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(inner_optimizer, T_0=math.ceil(n_inner_step/4), T_mult=2, eta_min=0.001)
for i in range(n_inner_step):
    print(f'[{i}] Before = {z_prime[0, 0, :5].detach().numpy().round(4)} lr={inner_optimizer.state_dict()["param_groups"][0]["lr"]:.4f}')
    inner_optimizer.zero_grad()
    # z_prime.retain_grad()
    s_loss.backward(retain_graph=True)
    inner_optimizer.step()
    inner_scheduler.step()
    # z_prime = z_prime - model.inner_lr * z_prime.grad.data
    print(f'[{i}] Grad = {z_prime.grad.data[0, 0, :5]}')
    print(f'[{i}] After = {z_prime[0, 0, :5].detach().numpy().round(4)} lr={inner_optimizer.state_dict()["param_groups"][0]["lr"]:.4f}')
    s_pred_loss, s_param_l2_loss, s_preds, parameters = model.forward_decoder(z=z_prime, l=s_l, labels=s_labels)
    s_loss = s_pred_loss + model.param_l2_lambda * s_param_l2_loss

print(f's_z: {s_z.requires_grad} | z_prime = {z_prime.requires_grad}')
print(f's_z: {s_z.is_leaf} | z_prime = {z_prime.is_leaf}')

[0] Before = [ 0.1054  0.5618  0.0916 -0.0534 -1.7327] lr=0.1000
[0] Grad = tensor([ 0.0032, -0.0019, -0.0013, -0.0027,  0.0047])
[0] After = [ 0.1051  0.562   0.0917 -0.0531 -1.7332] lr=0.0900
[1] Before = [ 0.1051  0.562   0.0917 -0.0531 -1.7332] lr=0.0900
[1] Grad = tensor([-0.0003, -0.0005,  0.0008, -0.0013,  0.0023])
[1] After = [ 0.1051  0.562   0.0916 -0.053  -1.7334] lr=0.0810
[2] Before = [ 0.1051  0.562   0.0916 -0.053  -1.7334] lr=0.0810
[2] Grad = tensor([-0.0022,  0.0015, -0.0014, -0.0004, -0.0030])
[2] After = [ 0.1053  0.5619  0.0917 -0.053  -1.7331] lr=0.0729
[3] Before = [ 0.1053  0.5619  0.0917 -0.053  -1.7331] lr=0.0729
[3] Grad = tensor([ 0.0019, -0.0026, -0.0023, -0.0048,  0.0032])
[3] After = [ 0.1052  0.5621  0.0919 -0.0526 -1.7334] lr=0.0656
[4] Before = [ 0.1052  0.5621  0.0919 -0.0526 -1.7334] lr=0.0656
[4] Grad = tensor([-4.9602e-04,  2.2589e-04,  3.7754e-04,  7.2105e-05, -1.6132e-03])
[4] After = [ 0.1052  0.5621  0.0919 -0.0526 -1.7333] lr=0.0590
[5] Before

In [67]:
z_prime = z_prime.detach()
z_loss = torch.mean((z_prime - s_z)**2)

print(f's_z: {s_z.requires_grad} | z_prime = {z_prime.requires_grad}')
print(f's_z: {s_z.is_leaf} | z_prime = {z_prime.is_leaf}')

s_z: True | z_prime = False
s_z: False | z_prime = True


---

## Universe

In [ ]:
import json 
import numpy as np

In [ ]:

def create_universe(seed, stock_names):
    stocks = {}
    np.random.seed(seed)
    all_idx = np.arange(len(ps))
    train_idx = np.random.choice(all_idx, size=(int(len(ps)*0.7)), replace=False)
    valid_test_idx = all_idx[~np.isin(all_idx, train_idx)]
    valid_idx = np.random.choice(valid_test_idx, size=(int(len(valid_test_idx)*(0.2/0.3))), replace=False)
    test_idx = valid_test_idx[~np.isin(valid_test_idx, valid_idx)]
    stocks['train'] = list(stock_names[train_idx])
    stocks['valid'] = list(stock_names[valid_idx])
    stocks['test'] = list(stock_names[test_idx])
    stocks['seed'] = seed
    return stocks

In [ ]:
ps = list((meta_train.data_dir / 'kdd17/price_long_50').glob('*.csv'))
stock_names = np.array([p.name.rstrip('.csv') for p in ps])
stocks = create_universe(seed=7, stock_names=stock_names)

In [ ]:
import json

with (meta_train.data_dir / 'kdd17'/ 'stock_universe.json').open('w') as file:
    json.dump(stocks, file)

In [ ]:
ps = list((meta_train.data_dir / 'stocknet-dataset/price/raw').glob('*.csv'))
stock_names = np.array([p.name.rstrip('.csv') for p in ps])
stocks = create_universe(seed=7, stock_names=stock_names)

with (meta_train.data_dir / 'stocknet-dataset'/ 'stock_universe.json').open('w') as file:
    json.dump(stocks, file)

-------

## Metrics